In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
batch_size = 1

# Hamburger Dataset
class TrainDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./vectors.csv',
            delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:15])
        self.target_data = torch.from_numpy(hamdata[:, 15:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len
    
    
class TestDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./tester.csv',
            delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:15])
        self.target_data = torch.from_numpy(hamdata[:, 15:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len
    
train_dataset = TrainDataset()
test_dataset = TestDataset()

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader =  torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [3]:
class NNModel(torch.nn.Module):
    def __init__(self):
        super(NNModel,self).__init__()
        self.l1 = nn.Linear(15, 32)
        self.l2 = nn.Linear(32, 32)
        self.l3 = nn.Linear(32, 32)
        self.l4 = nn.Linear(32, 32)
        self.l5 = nn.Linear(32, 10)
        
    def forward(self,x):
        # input data : ( n , 1 , 28 , 28 )
        x = x.view(-1, 15) # Flatten : ( n , 784 )
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [4]:
model = NNModel()


criterion = nn.L1Loss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.5)

model= model.to(device)

def train(epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)
        output = model(data)
        
        optimizer.zero_grad()

        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx%50==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.data))

In [5]:
def test():
    model.eval()
    test_loss=0
    correct=0
    for data,target in test_loader:
        data = data.to(device)
        target = target.to(device)

        output = model(data)
        test_loss += criterion(output,target).data
        pred = output
        # correct += pred.eq(target.data.view_as(pred)).sum()
        correct += abs(pred.sub(target.data.view_as(pred))).sum()
        
        print(test_loss)
        print(correct)
        test_loss /= len(test_loader.dataset)  
    print('Test set: Average loss: {:.9f}, Accuracy: {:.0f}%'.format(
        test_loss, 100. - (((correct)/len(output)))))


    '''
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset))) 
    '''

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/500 (0%)]	Loss: 0.096003
Train Epoch: 1 [50/500 (10%)]	Loss: 0.011831
Train Epoch: 1 [100/500 (20%)]	Loss: 0.004686
Train Epoch: 1 [150/500 (30%)]	Loss: 0.003626
Train Epoch: 1 [200/500 (40%)]	Loss: 0.003663
Train Epoch: 1 [250/500 (50%)]	Loss: 0.003927
Train Epoch: 1 [300/500 (60%)]	Loss: 0.004616
Train Epoch: 1 [350/500 (70%)]	Loss: 0.002558
Train Epoch: 1 [400/500 (80%)]	Loss: 0.003516
Train Epoch: 1 [450/500 (90%)]	Loss: 0.003450
tensor(0.0030)
tensor(0.0304, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(0.0575, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(0.0840, grad_fn=<AddBackward0>)
tensor(0.0032)
tensor(0.1162, grad_fn=<AddBackward0>)
tensor(0.0034)
tensor(0.1505, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(0.1788, grad_fn=<AddBackward0>)
tensor(0.0016)
tensor(0.1945, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(0.2144, grad_fn=<AddBackward0>)
tensor(0.0016)
tensor(0.2301, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(0.2528, grad_fn=<AddBackward0>)
tens

tensor(0.0023)
tensor(4.7178, grad_fn=<AddBackward0>)
tensor(0.0024)
tensor(4.7416, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(4.7616, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(4.7850, grad_fn=<AddBackward0>)
tensor(0.0029)
tensor(4.8143, grad_fn=<AddBackward0>)
tensor(0.0031)
tensor(4.8447, grad_fn=<AddBackward0>)
tensor(0.0035)
tensor(4.8796, grad_fn=<AddBackward0>)
tensor(0.0029)
tensor(4.9086, grad_fn=<AddBackward0>)
tensor(0.0024)
tensor(4.9324, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(4.9589, grad_fn=<AddBackward0>)
tensor(0.0031)
tensor(4.9902, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(5.0104, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(5.0300, grad_fn=<AddBackward0>)
tensor(0.0032)
tensor(5.0617, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(5.0885, grad_fn=<AddBackward0>)
tensor(0.0039)
tensor(5.1272, grad_fn=<AddBackward0>)
tensor(0.0014)
tensor(5.1415, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(5.1673, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(5.1851

tensor(8.6365, grad_fn=<AddBackward0>)
tensor(0.0029)
tensor(8.6655, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(8.6881, grad_fn=<AddBackward0>)
tensor(0.0045)
tensor(8.7327, grad_fn=<AddBackward0>)
tensor(0.0032)
tensor(8.7644, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(8.7927, grad_fn=<AddBackward0>)
tensor(0.0024)
tensor(8.8165, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(8.8429, grad_fn=<AddBackward0>)
tensor(0.0033)
tensor(8.8757, grad_fn=<AddBackward0>)
tensor(0.0040)
tensor(8.9159, grad_fn=<AddBackward0>)
tensor(0.0041)
tensor(8.9570, grad_fn=<AddBackward0>)
tensor(0.0019)
tensor(8.9756, grad_fn=<AddBackward0>)
tensor(0.0024)
tensor(8.9994, grad_fn=<AddBackward0>)
tensor(0.0029)
tensor(9.0284, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(9.0492, grad_fn=<AddBackward0>)
tensor(0.0016)
tensor(9.0653, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(9.0879, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(9.1138, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(9.1406, grad_fn=<AddB

Train Epoch: 2 [100/500 (20%)]	Loss: 0.002377
Train Epoch: 2 [150/500 (30%)]	Loss: 0.003868
Train Epoch: 2 [200/500 (40%)]	Loss: 0.001510
Train Epoch: 2 [250/500 (50%)]	Loss: 0.003818
Train Epoch: 2 [300/500 (60%)]	Loss: 0.002793
Train Epoch: 2 [350/500 (70%)]	Loss: 0.001316
Train Epoch: 2 [400/500 (80%)]	Loss: 0.002179
Train Epoch: 2 [450/500 (90%)]	Loss: 0.001194
tensor(0.0041)
tensor(0.0412, grad_fn=<AddBackward0>)
tensor(0.0033)
tensor(0.0741, grad_fn=<AddBackward0>)
tensor(0.0013)
tensor(0.0866, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(0.1071, grad_fn=<AddBackward0>)
tensor(0.0033)
tensor(0.1398, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(0.1660, grad_fn=<AddBackward0>)
tensor(0.0033)
tensor(0.1989, grad_fn=<AddBackward0>)
tensor(0.0012)
tensor(0.2107, grad_fn=<AddBackward0>)
tensor(0.0025)
tensor(0.2352, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(0.2579, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(0.2862, grad_fn=<AddBackward0>)
tensor(0.0035)
tensor(0.3207, grad_fn=

tensor(0.0026)
tensor(4.7746, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(4.8017, grad_fn=<AddBackward0>)
tensor(0.0038)
tensor(4.8395, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(4.8673, grad_fn=<AddBackward0>)
tensor(0.0013)
tensor(4.8798, grad_fn=<AddBackward0>)
tensor(0.0041)
tensor(4.9210, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(4.9438, grad_fn=<AddBackward0>)
tensor(0.0032)
tensor(4.9758, grad_fn=<AddBackward0>)
tensor(0.0013)
tensor(4.9889, grad_fn=<AddBackward0>)
tensor(0.0016)
tensor(5.0053, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(5.0255, grad_fn=<AddBackward0>)
tensor(0.0036)
tensor(5.0617, grad_fn=<AddBackward0>)
tensor(0.0015)
tensor(5.0767, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(5.1043, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(5.1255, grad_fn=<AddBackward0>)
tensor(0.0024)
tensor(5.1493, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(5.1676, grad_fn=<AddBackward0>)
tensor(0.0030)
tensor(5.1972, grad_fn=<AddBackward0>)
tensor(0.0012)
tensor(5.2089

tensor(0.0020)
tensor(10.0972, grad_fn=<AddBackward0>)
tensor(0.0015)
tensor(10.1122, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(10.1318, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(10.1523, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(10.1782, grad_fn=<AddBackward0>)
tensor(0.0033)
tensor(10.2111, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(10.2294, grad_fn=<AddBackward0>)
tensor(0.0024)
tensor(10.2532, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(10.2715, grad_fn=<AddBackward0>)
tensor(0.0012)
tensor(10.2833, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(10.3037, grad_fn=<AddBackward0>)
tensor(0.0034)
tensor(10.3380, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(10.3590, grad_fn=<AddBackward0>)
tensor(0.0034)
tensor(10.3934, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(10.4146, grad_fn=<AddBackward0>)
tensor(0.0017)
tensor(10.4316, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(10.4576, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(10.4788, grad_fn=<AddBackward0>)
tensor(0.0

tensor(2.2126, grad_fn=<AddBackward0>)
tensor(0.0017)
tensor(2.2297, grad_fn=<AddBackward0>)
tensor(0.0014)
tensor(2.2441, grad_fn=<AddBackward0>)
tensor(0.0015)
tensor(2.2590, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(2.2771, grad_fn=<AddBackward0>)
tensor(0.0017)
tensor(2.2940, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(2.3217, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(2.3425, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(2.3698, grad_fn=<AddBackward0>)
tensor(0.0017)
tensor(2.3866, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(2.4125, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(2.4333, grad_fn=<AddBackward0>)
tensor(0.0017)
tensor(2.4506, grad_fn=<AddBackward0>)
tensor(0.0022)
tensor(2.4730, grad_fn=<AddBackward0>)
tensor(0.0022)
tensor(2.4954, grad_fn=<AddBackward0>)
tensor(0.0026)
tensor(2.5215, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(2.5424, grad_fn=<AddBackward0>)
tensor(0.0017)
tensor(2.5597, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(2.5779, grad_fn=<AddB

tensor(6.5226, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(6.5406, grad_fn=<AddBackward0>)
tensor(0.0031)
tensor(6.5714, grad_fn=<AddBackward0>)
tensor(0.0032)
tensor(6.6035, grad_fn=<AddBackward0>)
tensor(0.0016)
tensor(6.6198, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(6.6379, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(6.6561, grad_fn=<AddBackward0>)
tensor(0.0023)
tensor(6.6790, grad_fn=<AddBackward0>)
tensor(0.0031)
tensor(6.7098, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(6.7296, grad_fn=<AddBackward0>)
tensor(0.0028)
tensor(6.7578, grad_fn=<AddBackward0>)
tensor(0.0029)
tensor(6.7863, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(6.8066, grad_fn=<AddBackward0>)
tensor(0.0020)
tensor(6.8264, grad_fn=<AddBackward0>)
tensor(0.0016)
tensor(6.8427, grad_fn=<AddBackward0>)
tensor(0.0027)
tensor(6.8693, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(6.8868, grad_fn=<AddBackward0>)
tensor(0.0018)
tensor(6.9045, grad_fn=<AddBackward0>)
tensor(0.0021)
tensor(6.9253, grad_fn=<AddB